### Imports

In [0]:
from pyspark.sql.functions import (
    col, sum as _sum, filter, when,concat, coalesce, lit, trim, expr, substring, locate,date_format,count,collect_set,concat_ws,countDistinct,
     lower, rtrim, split, regexp_extract, regexp_replace, array_max)
import re
import seaborn as sns
import sys
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

spark.conf.set("spark.sql.parquet.datetimeRebaseModeInRead", "LEGACY") # Spark configuration for Parquet datetime handling

### Raw Data

In [0]:
def load_dataset(table_names, dataframe_names):
    df_dict={}
    for i in range(len(table_names)):
        df_dict[dataframe_names[i]] = spark.table(table_names[i])
        globals().update(df_dict)

In [0]:
table_names=["lh_imacs.EA_Project_ActualHours","lh_imacs.EA_Project_BudgetEACHours"]
dataframe_names = ["df_project_actualHours","df_Project_BudgetEACHours"]

if len(table_names) != len(dataframe_names):
    raise ValueError("Mismatch between table names and dataframe names.")

try:
    load_dataset(table_names, dataframe_names)
except BaseException as exception:
    exception_type, exception_object, exception_traceback = sys.exc_info()
    print(f"Exception: {exception_type.__name__}")
    print(f"\nError on line {exception_traceback.tb_lineno}")
    print(f"\nException message: {exception}")


In [0]:
display(df_project_actualHours.count())
display(df_project_actualHours)
display(df_Project_BudgetEACHours.count())
display(df_Project_BudgetEACHours)


In [0]:
project_actualHours_columns = ['StockNo', 'TypeCode', 'Hours','ServiceSegment','ServiceDate']

# Convert 'ServiceDate' to datetime
df_project_actualHours_filter = (df_project_actualHours
    .select(project_actualHours_columns)
    .withColumn('ServiceDate', F.to_date('ServiceDate'))
    .withColumn('ServiceMonthYear', date_format(col('ServiceDate'), 'yyyy-MM'))
    .withColumn("StockNo", trim(lower(col("StockNo"))))
    .withColumn("ServiceSegment", trim(lower(col("ServiceSegment"))))
    )

display(df_project_actualHours_filter.count())

Project_BudgetEACHours_columns = ['StockNo', 'ProjectNo','SellPriceCAD','IndustryCode','Model','Stocktype']

df_Project_BudgetEACHours_filter =(df_Project_BudgetEACHours
    .select(Project_BudgetEACHours_columns)
    .withColumn("StockNo", trim(lower(col("StockNo"))))
    .withColumn("ProjectNo", trim(lower(col("ProjectNo"))))
    .withColumn("IndustryCode", trim(lower(col("IndustryCode"))))
    .withColumn("Model", trim(lower(col("Model"))))
    .withColumn("Stocktype", trim(lower(col("Stocktype"))))
    )

display(df_Project_BudgetEACHours_filter.count())

In [0]:
df_add_projectno_to_actual = (df_project_actualHours_filter
    .join(df_Project_BudgetEACHours_filter,"StockNo","left")
    .filter(col("ProjectNo").isNotNull())
    .drop("ServiceDate")
    )
display(df_add_projectno_to_actual.count())
display(df_add_projectno_to_actual)

df_add_projectno_to_actual = df_add_projectno_to_actual.withColumn('2020_ind', F.when((col('ServiceMonthYear')>= '2020-01'), 1).otherwise(0))

# comment this out to remove date filter
# df_add_projectno_to_actual = df_add_projectno_to_actual.filter(col('ServiceMonthYear')>='2020-01')
display(df_add_projectno_to_actual.count())



In [0]:
df_2020_ind = df_add_projectno_to_actual.groupBy('ProjectNo').agg(F.median('2020_ind').alias('2020_ind'))

In [0]:
stock_count_by_project = (
    df_add_projectno_to_actual
    .groupBy("ProjectNo")
    .agg(
        count("StockNo").alias("stock_count"),
        countDistinct("model").alias("UniqueModelCount"),
        _sum("Hours").alias("hours"),
        _sum("SellPriceCAD").alias("total_sell_price"),
        concat_ws(",", collect_set("IndustryCode")).alias("industry_codes"))
)

# remove big outliers
#stock_count_by_project = stock_count_by_project.filter(col('total_sell_price')<= 1E7)

stock_count_by_project.count()

In [0]:
display(stock_count_by_project)

In [0]:
df_with_ind = stock_count_by_project.join(df_2020_ind, 'ProjectNo', 'left')
display(df_with_ind)

In [0]:
import matplotlib.pyplot as plt
df_pd = df_with_ind.toPandas()

df_pd['total_sell_price'].apply(pd.to_numeric).plot.box()
plt.ylabel("total_sell_rpice")
plt.title("total_sell_price Box Plot")
plt.show()


In [0]:
display(df_with_ind.sort('total_sell_price', ascending = False))

In [0]:
# stock_count_by_project = (
#     df_typecode_stocktype
#     .groupBy("ProjectNo")
#     .agg(
#         count("StockNo").alias("stock_count"),
#         countDistinct("model").alias("UniqueModelCount"),
#         _sum("Hours").alias("hours"),
#         # _sum("genset_ct").alias("genset_ct"),
#         # _sum('other_ct').alias('other_ct'),
#         # _sum('ats_ct').alias('ats_ct'),
#         # _sum('ctrl_panel_ct').alias('ctrl_panel_ct'),
#         # _sum('switchgear_ct').alias('switchgear_ct'),
#         # _sum('engine_ct').alias('engine_ct'),
#         _sum("SellPriceCAD").alias("total_sell_price"),
#         concat_ws(",", collect_set("IndustryCode")).alias("industry_codes"))
# )
# stock_count_by_project.count()

In [0]:
import statsmodels.api as sm 
import pylab as py 
import numpy as np
import seaborn as sns
  
sm.qqplot(df_with_ind.toPandas()['hours'], line ='45') 
py.show() 

sns.displot(df_with_ind.toPandas()['hours'])

In [0]:
from scipy import stats

df_bc = df_with_ind.toPandas()['hours']

df_col = stats.boxcox(df_bc)[0]

df_transformed_pd = df_with_ind.toPandas()
df_transformed_pd['hours'] = df_col

df_transformed= spark.createDataFrame(df_transformed_pd)

In [0]:
display(df_transformed)

In [0]:
# sm.qqplot(df_transformed_col, line ='45') 
# py.show() 

sns.displot(df_col)

In [0]:
df_with_ind.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable("data")


In [0]:
filepath=["dbfs:/FileStore/Imacs/df_data_21.csv"]
raw_df = spark.read.csv(filepath, sep=",", header=True, inferSchema=True)
raw_df = (
    raw_df
    .drop('ProjectNo', 'industry_codes')
    .filter(col('total_sell_price') <= 1E8)
    .filter(col('hours') > 0)
    .filter(col('hours') < 1500)
    .filter(col('stock_count') < 500)
)
target_col = "hours"

In [0]:
raw_pd = raw_df.toPandas()

In [0]:
df_transformed = raw_pd1.copy()
df_transformed['transformed_hours'], lmda = stats.boxcox(df_transformed['hours'])

# # validation check
# # df_transformed['hours_check'] = inv_boxcox(df_transformed['transformed_hours'], lmda)
# # display(df_transformed)

raw_pd['hours'] = df_transformed['transformed_hours']

raw_df = spark.createDataFrame(raw_pd)

raw_df.count()
